<a href="https://colab.research.google.com/github/Hojo0210/finalProject/blob/CarsonWBr/S3toPostGres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-09-16 16:04:42--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.2’

postgresql-42.2.16. 100%[===================>] 979.38K  5.62MB/s    in 0.2s    

2021-09-16 16:04:43 (5.62 MB/s) - ‘postgresql-42.2.16.jar.2’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://cfwest-mldata.s3.amazonaws.com/Census_Counts_Cleaned.csv"
spark.sparkContext.addFile(url)
population_df = spark.read.csv(SparkFiles.get("Census_Counts_Cleaned.csv"), sep=",", header=True, inferSchema=True)


In [ ]:
population_df.distinct().count()

1340

In [ ]:
url ="https://cfwest-mldata.s3.amazonaws.com/Zillow_Master_cleaned.csv"
spark.sparkContext.addFile(url)
housing_df_cleaned = spark.read.csv(SparkFiles.get("Zillow_Master_cleaned.csv"), sep=",", header=True, inferSchema=True)

In [ ]:
housing_df_cleaned.distinct().count()

1340

In [ ]:
url ="https://cfwest-mldata.s3.amazonaws.com/Rainfall_Florida_Counties_Yearly.csv"
spark.sparkContext.addFile(url)
rainfall_df = spark.read.csv(SparkFiles.get("Rainfall_Florida_Counties_Yearly.csv"), sep=",", header=True, inferSchema=True)

In [ ]:
rainfall_df.distinct().count()

1340

In [ ]:
url ="https://cfwest-mldata.s3.amazonaws.com/Wage_data_by_county_20yr.csv"
spark.sparkContext.addFile(url)
wages_df = spark.read.csv(SparkFiles.get("Wage_data_by_county_20yr.csv"), sep=",", header=True, inferSchema=True)

In [ ]:
wages_df.distinct().count()

1340

In [4]:
from pyspark import SparkFiles
url ="https://cfwest-mldata.s3.amazonaws.com/Florida_Crime.csv"
spark.sparkContext.addFile(url)
crime_df = spark.read.csv(SparkFiles.get("Florida_Crime.csv"), sep=",", header=True, inferSchema=True)

In [5]:
crime_df.distinct()

DataFrame[county: string, Year: int, Murder: int, Rape: int, Robbery: int, Aggravated_assault: int, Burglary: int, Larceny: int, Motor_vehicle_theft: int, Index_per_100k: int]

In [6]:
url ="https://cfwest-mldata.s3.amazonaws.com/employment_cleaned.csv"
spark.sparkContext.addFile(url)
employment_df = spark.read.csv(SparkFiles.get("employment_cleaned.csv"), sep=",", header=True, inferSchema=True)

In [7]:
employment_df.distinct().count()

1340

In [7]:
# Store environmental variable
from getpass import getpass
password = getpass('postgres')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://mldata.chbvxx9zicoa.us-east-2.rds.amazonaws.com:5432/mldata"
config = {"user":"postgres",
          "password": password,
          "driver":"org.postgresql.Driver"}

postgres··········


In [ ]:
# Write DataFrame to active_user table in RDS
population_df.write.jdbc(url=jdbc_url, table='Population', mode=mode, properties=config)

In [ ]:
housing_df_cleaned.write.jdbc(url=jdbc_url, table='Housing', mode=mode, properties=config)

In [ ]:
rainfall_df.write.jdbc(url=jdbc_url, table='Rainfall', mode=mode, properties=config)

In [ ]:
wages_df.write.jdbc(url=jdbc_url, table='Wages', mode=mode, properties=config)

In [8]:
crime_df.write.jdbc(url=jdbc_url, table='Crime', mode=mode, properties=config)

In [9]:
employment_df.write.jdbc(url=jdbc_url, table='Employment', mode=mode, properties=config)